<a href="https://colab.research.google.com/github/Ratna-Babu/Handwritten-Digit-Classification-using-CNN/blob/main/Handwritten_Digit_Classification_usingCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Invert the colors of the images
x_train = 255 - x_train
x_test = 255 - x_test


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
# Preprocessing the Images
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape the images to include the channel dimension
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.fit(x_train, y_train, epochs=10, batch_size=200, validation_split=0.2)
model.save('mnist_cnn.h5')

Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 42s 167ms/step - accuracy: 0.7631 - loss: 0.8423 - val_accuracy: 0.9724 - val_loss: 0.0955
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 39s 162ms/step - accuracy: 0.9729 - loss: 0.0858 - val_accuracy: 0.9795 - val_loss: 0.0687
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 41s 163ms/step - accuracy: 0.9834 - loss: 0.0571 - val_accuracy: 0.9854 - val_loss: 0.0514
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 40s 165ms/step - accuracy: 0.9873 - loss: 0.0445 - val_accuracy: 0.9861 - val_loss: 0.0473
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 40s 165ms/step - accuracy: 0.9887 - loss: 0.0360 - val_accuracy: 0.9862 - val_loss: 0.0443
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 39s 162ms/step - accuracy: 0.9924 - loss: 0.0263 - val_accuracy: 0.9861 - val_loss: 0.0488
Epoch 7/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 42s 168ms/step - accuracy: 0.9928 - loss: 0.0233 - val_accuracy: 0.9869 - val_loss: 0.0406
Epoch 8/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 42s 172ms/step - accuracy: 0.9937 - loss: 0

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {score[1]:.4f}')

Test accuracy: 0.9900


In [7]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the trained model
model = load_model('mnist_cnn.h5')

def predict_digit(image_path):
    try:
        # Load and preprocess the image
        img = Image.open(image_path).convert('L')
        img = img.resize((28, 28))
        img_array = np.array(img)
        img_array = 255 - img_array
        img_array = img_array.astype('float32') / 255.0
        img_array = img_array.reshape(1, 28, 28, 1)

        # Make the prediction
        prediction = model.predict(img_array)
        predicted_digit = np.argmax(prediction)
        confidence = np.max(prediction)

        display_prediction(image_path, predicted_digit, confidence)

        return predicted_digit

    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def display_prediction(image_path, digit, confidence):
    img = cv2.imread(image_path)

    img_resized = cv2.resize(img, (500, 500))

    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

    label = f'Predicted: {digit} ({confidence:.2f})'
    cv2.putText(img_rgb, label, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

image_path = '2.png'  # Replace with the actual path to your image
predicted_digit = predict_digit(image_path)

if predicted_digit is not None:
    print(f"Predicted digit: {predicted_digit}")


Error: Image file not found at 2.png
